# To save surface wind diagnostics into CSV files

In [ ]:
from ipynb.fs.full.procc import *
from statsmodels.nonparametric.smoothers_lowess import lowess
import seaborn as sns
from scipy.stats import sem, ttest_ind
import matplotlib.cm as cm
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import sys
from scipy.signal import savgol_filter
import csv

### only wanted binnedDP, do original DP (0.005) and check 0.04? (maybe should just recalc everything with 0.04 too?)
### change output location, i.e. + "binnedDP/"

## Function to process files into a dictionary

In [ ]:
# Vars = [U, V, W, Dir, DP, DP_STD, DP_STDERR, RDD]
# wind direction 'to'/towards degree, clockwise from north = 0°

def procFiles(for_direc, var, average):
    
    sorted_file_dict = {}
    file_dict = {}
    file_end = var+".csv"
    
    #creates a dictionary with the file name as the key and the path of the file as the value
    for root, dirs, filenames in os.walk(for_direc):
        for file in filenames:
            if average:
                if file.endswith(file_end) and 'Avg' in file: 
                    file_to_add = os.path.join(root, file) 
                    file_dict[file] = file_to_add
            else: 
                if file.endswith(file_end) and 'Avg' not in file: 
                    file_to_add = os.path.join(root, file) 
                    file_dict[file] = file_to_add
    
    for key in sorted(file_dict.keys()):
        sorted_file_dict[key] = file_dict[key]
        
    return sorted_file_dict

In [ ]:
def openCSV(fn):
    array = []
    
    with open(fn, newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in spamreader:
            for num in row:
                array.append(float(num))
    return array

In [ ]:
def direcToArray(direc):
    arr = []
    
    if len(direc.keys()) > 1:
        for key in direc.keys():
            arr.append(openCSV(direc[key]))
    else:
        key = list(direc.keys())
        key=key[0]
        arr = openCSV(direc[key])
   
    return arr

In [ ]:
def assignDay(arr):
    arrDay = []
    
    for i in range(0,int(len(arr)/766)):
        arrDay.append(arr[766*i:766*(i+1)])
    
    return arrDay

In [ ]:
def flattenLoL(arr): #flatten list of lists
    flat_arr = []
    
    for list in arr:
        for val in list:
            flat_arr.append(val)

    return flat_arr

## Assign constants and obtain input parameters

In [ ]:
# fensal eastern edge = 2.0 lat, 2.5 lon
# fensal center = 2.0 lat, - 52.5 lon
# shangrila eastern edge = -6.0 lat, -142.5
# shangrila center = -6.0 lat, -177.5 lon
# belet eastern edge = 2.0 lat, 142.5 lon
# belet center = 2.0 lat, 107.5 lon

#UT = [uf*24.3 for uf in UF] # 24.3 from log wind profile equation

In [ ]:
direc = str(sys.argv[1]) # Ends in '/'
run = str(sys.argv[2].upper())
region = str(sys.argv[3].lower()) 
location = str(sys.argv[4].lower())
average = sys.argv[5].lower() == 'true'
UF = float(sys.argv[6])

In [ ]:
loc_name = ''

if region == 'belet':
    region = '/Belet/'
    reg_name = 'Belet'
elif region == 'shangrila':
    region = '/ShangriLa/'
    reg_name = 'Shang'
elif region == 'fensal':
    region = '/Fensal/'
    reg_name = 'Fen'
elif region == 'equator_timestep':
    region = '/Equator/Timestep/'
    reg_name = 'Eq'
    location = ''

if region == 'equator_timestep' or region == 'equator_daily':
    pass
elif location == 'center':
    location = 'Center/'
    loc_name = 'C'
elif location == 'edge':
    location = 'EastEdge/'
    loc_name = 'E'

name = run + "_" + reg_name + loc_name + "_"
direc = direc + run + region + location
saveloc = direc + "BinnedDP/"

In [ ]:
W_Direc = procFiles(direc,'W', average)
Dir_Direc = procFiles(direc,'Dir', average)

In [ ]:
W = direcToArray(W_Direc)
DIR = direcToArray(Dir_Direc)

In [ ]:
if not average:
    W = flattenLoL(W)
    DIR = flattenLoL(DIR)

    W = assignDay(W)
    DIR = assignDay(DIR)

In [ ]:
if average:
    name = name + 'Avg'
else:
    name = name + 'All'
    
savename = name+"_"+str(UF).replace(".","")

In [ ]:
output = saveloc+savename

In [ ]:
def binnedDP(W, Dir, UT):
           
    bins=[]
    windbins = np.arange(0,361,22.5)
    deg_midP = [(windbins[i]+windbins[i+1])/2 for i in range(0,len(windbins)-1)]

    for i in range(0,len(windbins)-1):
        if i == len(windbins)-1:
            bins.append((xr.where((Dir>=windbins[i]) & (Dir<=windbins[i+1]), True, False)))
        else:
            bins.append((xr.where((Dir>=windbins[i]) & (Dir<windbins[i+1]), True, False)))


    t = xr.where(W>UT,1,0)
    total=len(t)
    cum_sum = sum(t)
    t = float(cum_sum/total)

    bins = [(np.ma.masked_equal((bins[i]*W),0)).compressed() for i in range(0,len(bins))]

    binnedWavg = [np.mean(bins[i], axis=0) for i in range(0,len(bins))]

    binnedDP = [(binnedWavg[i]**2 * (binnedWavg[i] - UT) * t) / 100. for i in range(0,len(bins))]

    binnedDP = [dp if dp >= 0.0 else 0.0 for dp in binnedDP]
    
    bin_DP = [(binnedDP[i], deg_midP[i]) for i in range(0,len(binnedDP))]
    
    return bin_DP

    #     # For calculation of threshold velocity at 10 m, need to figure out if right
    #     # https://books.google.com.hk/books?hl=en&lr=&id=eaxTAVp9avkC&oi=fnd&pg=PA137&ots=5nAJ__p7CX&sig=_a4lg7V3Io9mrdCbMY3I7uW0HD4&redir_esc=y#v=onepage&q=threshold&f=false

In [ ]:
UT = UF*24.3

allBinDP = []

for day, w in enumerate(W):
    binDP = binnedDP(np.array(w),np.array(DIR[day]),UT)
    to_del = []
    
    for index, dp in enumerate(binDP):
        if dp[0] == 0.0:
            to_del.append(index)
    binDP_clean = np.delete(binDP, to_del, axis=0)

    allBinDP.append(binDP_clean)

allBinDP = flattenLoL(allBinDP)

DP = np.take(allBinDP, 0, axis=1)
DIR = np.take(allBinDP, 1, axis=1)

np.savetxt(output+"_binDP.csv", DP, delimiter=",")
np.savetxt(output+"_binDIR.csv", DIR, delimiter=",")

In [ ]:
# import argparse

# # defined command line options
# # this also generates --help and error handling

# CLI=argparse.ArgumentParser()

# CLI.add_argument(
#   "--loc",  # name on the CLI - drop the `--` for positional/required parameters
#   nargs="*",  # 0 or more values expected => creates a list
#   type=tuple,
#   default=[('-2','-2.5'),('-2','2.5'),('2','2.5'),('2','-2.5')],  # default if nothing is provided
# )


# # parse the command line
# args = CLI.parse_args()
# # access CLI options
# print("loc: %r" % args.loc)

# loc = args.loc